# Imports

In [7]:
import numpy as np
import pandas as pd
import nagisa
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

## Read in and process data

In [2]:
wiki_df = pd.read_pickle('wiki') 
wiki_df.head()

,article,views,text
0,メインページ,362562853,ようこそ\nウィキペディア - ウィキペディア日本語版 - 百科事典目次\n検索資料・ポータ...
1,星野源,10190763,星野 源（ほしの みなもと、1981年1月28日 - ）は、日本の音楽家、俳優、文筆家。埼玉...
2,真田信繁,9602104,真田 信繁（さなだ のぶしげ）は、安土桃山時代から江戸時代初期にかけての武将、大名。真田昌幸...
3,高橋一生,8571666,高橋 一生（たかはし いっせい、英字表記：Issey Takahashi、1980年12月9...
4,君の名は。,7788879,『君の名は。』（きみのなは、英: Your Name.）は、2016年に公開された新海誠監督...


In [3]:
data = np.array(wiki_df['text'])

# Generate nagisa labeled data

In [ ]:
# Split each article using sentence delimiter indicators
data_split = []
for article in data:
    article_split = re.split('\n|\.|!|！|。', article)
    data_split.append(article_split)

In [8]:
labeled_data = []
# first ~700 due to data/time limitations
for doc_num in range(1000):
    for sent_num in range(len(data_split[doc_num])):
        cur_text = nagisa.tagging(data_split[doc_num][sent_num])
        words = cur_text.words
        postags = cur_text.postags
        char_offset = 0
        for i in range(len(words)):
            labeled_data.append([doc_num, sent_num, char_offset, words[i], postags[i]])
            char_offset += 1
    print("finished " + str(doc_num))
labeled_df = pd.DataFrame(labeled_data)
labeled_df.to_pickle('labeledDataNew.pkl')

finished 0
finished 1
finished 2
finished 3
finished 4
finished 5
finished 6
finished 7
finished 8
finished 9
finished 10
finished 11
finished 12
finished 13
finished 14
finished 15
finished 16
finished 17
finished 18
finished 19
finished 20
finished 21
finished 22
finished 23
finished 24
finished 25
finished 26
finished 27
finished 28
finished 29
finished 30
finished 31
finished 32
finished 33
finished 34
finished 35
finished 36
finished 37
finished 38
finished 39
finished 40
finished 41
finished 42
finished 43
finished 44
finished 45
finished 46
finished 47
finished 48
finished 49
finished 50
finished 51
finished 52
finished 53
finished 54
finished 55
finished 56
finished 57
finished 58
finished 59
finished 60
finished 61
finished 62
finished 63
finished 64
finished 65
finished 66
finished 67
finished 68
finished 69
finished 70
finished 71
finished 72
finished 73
finished 74
finished 75
finished 76
finished 77
finished 78
finished 79
finished 80
finished 81
finished 82
finished 83
fi

finished 639
finished 640
finished 641
finished 642
finished 643
finished 644
finished 645
finished 646
finished 647
finished 648
finished 649
finished 650
finished 651
finished 652
finished 653
finished 654
finished 655
finished 656
finished 657
finished 658
finished 659
finished 660
finished 661
finished 662
finished 663
finished 664
finished 665
finished 666
finished 667
finished 668
finished 669
finished 670
finished 671
finished 672
finished 673
finished 674
finished 675
finished 676
finished 677
finished 678
finished 679
finished 680
finished 681
finished 682
finished 683
finished 684
finished 685
finished 686
finished 687
finished 688
finished 689
finished 690
finished 691
finished 692
finished 693
finished 694
finished 695
finished 696
finished 697
finished 698
finished 699
finished 700
finished 701
finished 702
finished 703
finished 704
finished 705
finished 706
finished 707
finished 708
finished 709
finished 710
finished 711
finished 712
finished 713
finished 714
finished 715

KeyboardInterrupt: 

In [10]:
labeled_df = pd.DataFrame(labeled_data)

In [11]:
labeled_df.columns = ['docNum','sentNum','charOffset','word','posTag']

In [12]:
labeled_df.to_pickle('labeledDataNew.pkl')

In [5]:
labeled_df = pd.read_pickle('labeledDataNew.pkl')

In [6]:
labeled_df

,docNum,sentNum,charOffset,word,posTag
0,0,0,0,よう,形容詞
1,0,0,1,こそ,助詞
2,0,1,0,ウィキペディア,名詞
3,0,1,1,,空白
4,0,1,2,-,補助記号
...,...,...,...,...,...
9909358,771,1300,18,を,助詞
9909359,771,1300,19,受け,動詞
9909360,771,1300,20,て,助詞
9909361,771,1300,21,回復,名詞


# SVD of TFIDF + K-Means

https://hackernoon.com/advanced-topic-modeling-tutorial-how-to-use-svd-and-nmf-in-python-to-find-topics-in-text

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py

In [7]:
# Create space delimited text using nagisa output
space_delimited = []
for i in range(len(data)):
    concat_doc = ' '.join(list(labeled_df[(labeled_df['docNum']==i)]['word']))
    space_delimited.append(concat_doc)

In [12]:
# Vectorize and perform SVD
stop = ['あそこ','あっ','あの','あのかた','あの人','あり','あります','ある','あれ','い','いう','います','いる','う','うち','え','お','および','おり','おります','か','かつて','から','が','き','ここ','こちら','こと','この','これ','これら','さ','さらに','し','しかし','する','ず','せ','せる','そこ','そして','その','その他','その後','それ','それぞれ','それで','た','ただし','たち','ため','たり','だ','だっ','だれ','つ','て','で','でき','できる','です','では','でも','と','という','といった','とき','ところ','として','とともに','とも','と共に','どこ','どの','な','ない','なお','なかっ','ながら','なく','なっ','など','なに','なら','なり','なる','なん','に','において','における','について','にて','によって','により','による','に対して','に対する','に関する','の','ので','のみ','は','ば','へ','ほか','ほとんど','ほど','ます','また','または','まで','も','もの','ものの','や','よう','より','ら','られ','られる','れ','れる','を','ん','何','及び','彼','彼女','我々','特に','私','私達','貴方','貴方方''ようこそ','ウィキペディア','ウィキペディア日本語版','百科事典目次','検索資料','空白','補助記号','）', '（', '、', '『', '』', '・','：', '／', '＋', '→', '「', '」', ' ', '-', '/', '\\']
vectorizer = TfidfVectorizer(stop_words=stop, smooth_idf=True)
svdfit = vectorizer.fit_transform(space_delimited[0:500]) #first 500 documents
input_matrix = svdfit.todense()

svd_modeling = TruncatedSVD(n_components=15, algorithm='randomized', n_iter=100, random_state=3)
svd_out = svd_modeling.fit(input_matrix)
components = svd_modeling.components_
vocab = vectorizer.get_feature_names()

C:\Users\Marisa\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
C:\Users\Marisa\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Topic modeling using SVD alone

In [15]:
topic_word_list = []
def get_topics(components): 
    for i, comp in enumerate(components):
        terms_comp = zip(vocab,comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
        topic = " "
        for t in sorted_terms:
            topic= topic + ' ' + t[0]
        topic_word_list.append(topic)
    return topic_word_list

In [16]:
get_topics(components)

['  テレビ 主演 日本 フジ ドラマ nhk 出演',
 '  cd アニメ 発売 世界 よる 魔法 よっ',
 '  cd シリーズ ドラマ vol ガンダム ラジオ 声優',
 '  家康 真田 秀吉 徳川 豊臣 三成 ドラマ',
 '  家康 真田 秀吉 徳川 豊臣 三成 信繁',
 '  投手 本塁 安打 三振 試合 登板 失点',
 '  投手 cd 本塁 安打 三振 日本 登板',
 '  主演 ライダー 仮面 映画 女優 東宝 助演',
 '  ライダー 仮面 ライダーフォーゼ 戦隊 ドライブ テレビ 泊進ノ介',
 '  ルフィ 海賊 ドイツ ドフラミンゴ テレビ サンジ フジ',
 '  ドイツ アメリカ イギリス テレビ 日本 共和 人民',
 '  ジャニーズ 主演 smap ジャニー コナン 事件 hey',
 '  ルフィ 放送 海賊 ジャニーズ ウォーズ コナン ルーカス',
 '  井伊 直虎 井伊谷 今川 直親 直政 道好',
 '  将棋 対局 棋士 羽生 ジャニーズ 名人 藤井']

## Topic modeling using K-Means on SVD output

In [18]:
# Document clustering
# Uses output of the vectorizer and SVD above

svd_out = svd_modeling.fit_transform(input_matrix)
km = KMeans(n_clusters=20, init="k-means++", max_iter=100)
km.fit(svd_out)

print("Top terms per cluster:")

original_space_centroids = svd_modeling.inverse_transform(km.cluster_centers_)
order_centroids = original_space_centroids.argsort()[:, ::-1]

terms = vectorizer.get_feature_names_out()
for i in range(20):
    print("Cluster %d:" % i, end="")
    for ind in order_centroids[i, :10]:
        print(" %s" % terms[ind], end="")
    print()

C:\Users\Marisa\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Top terms per cluster:
Cluster 0: テレビ 主演 フジ ドラマ nhk tbs 日本 映画 1日 出演
Cluster 1: 日本 ドイツ アメリカ 民主 中国 政府 世界 人民 経済 問題
Cluster 2: 放送 日本 アニメ テレビ isbn 発売 登場 魔法 映画 よる
Cluster 3: アルバム シングル ライブ 発売 ツアー 公演 tour 出演 開催 リリース
Cluster 4: 家康 真田 秀吉 徳川 信繁 豊臣 三成 大坂 昌幸 大名
Cluster 5: cd シリーズ ドラマ ラジオ アニメ vol テレビ 魔法 声優 ガンダム
Cluster 6: 投手 本塁 安打 三振 登板 試合 失点 先発 記録 リーグ
Cluster 7: ライダー 仮面 主演 テレビ ライダーフォーゼ ドライブ 戦隊 泊進ノ介 シリーズ 弦太朗
Cluster 8: 放送 コナン アニメ テレビ 監督 作品 nhk 番組 ウォーズ 映画
Cluster 9: cd シリーズ ドラマ vol ラジオ ガンダム 声優 dvd the アニメ
Cluster 10: ルフィ 海賊 ドフラミンゴ サンジ 麦わら マム piece 能力 魚人 ドレスローザ
Cluster 11: 井伊 直虎 今川 井伊谷 家康 直親 直政 徳川 道好 直盛
Cluster 12: 議員 自民 内閣 安倍 選挙 大臣 民主 衆議 日本 政治
Cluster 13: ジャニーズ テレビ 日本 主演 シングル smap フジ アルバム 公演 出演
Cluster 14: ドイツ 日本 アメリカ イギリス 人民 世界 共和 中国 政府 朝鮮
Cluster 15: 将棋 対局 棋士 羽生 年度 名人 藤井 放送 加藤 優勝
Cluster 16: 能力 アニメ 登場 魔法 isbn よっ ナルト 事件 自分 発売
Cluster 17: テレビ フジ 日本 tbs 放送 出演 nhk 番組 朝日 1日
Cluster 18: 主演 テレビ 映画 ドラマ 女優 nhk フジ 日本 東宝 公開
Cluster 19: テレビ フジ 主演 日本 ドラマ tbs nhk 1日 出演 映画
